# Recommender with splitted user 

In [30]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
sys.path.append('..')

dataset = pd.read_csv('data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


In [31]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [32]:
items_per_user = np.ediff1d(URM_all.indptr) #tells in which position each of the rows begin
users_per_item = np.ediff1d(sps.csc_matrix(URM_all).indptr)
URM_all = sps.csr_matrix(URM_all)

In [33]:
items_per_user = np.sort(items_per_user) #sorting based on the single value. Losing reference to the user. 
users_per_item = np.sort(users_per_item)

In [34]:
ICM_df = pd.read_csv('data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [35]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [36]:
ICM_all = sps.csr_matrix(ICM_all)
features_per_item = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csc_matrix(ICM_all)
items_per_feature = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csr_matrix(ICM_all)

In [37]:
features_per_item = np.sort(features_per_item)
items_per_feature = np.sort(items_per_feature)

# Data pre-processing

In [38]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Base.NonPersonalizedRecommender import TopPop
from MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from KNN.ItemKNNScoresHybrid5Recommender import ItemKNNScoresHybrid5Recommender


from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender


from GraphBased.RP3betaRecommender_user import RP3betaRecommender_user
from MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender

from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

MAP_p3aplha_rp3beta_per_group = [0, 0, 0, 0, 0]
MAP_p3aplha_rp3beta_with_als_per_group = [0, 0, 0, 0, 0]
MAP_top_model_als_per_group = [0, 0, 0, 0, 0]
MAP_user_cf_cold_per_group = [0, 0, 0, 0, 0]
MAP_item_cf_cold_per_group = [0, 0, 0, 0, 0]
MAP_item_cbf_per_group = [0, 0, 0, 0, 0]
MAP_top_pop_per_group = [0, 0, 0, 0, 0]
MAP_pure_svd_per_group = [0, 0, 0, 0, 0]
MAP_slim_bpr_per_group = [0, 0, 0, 0, 0]
MAP_top_model_per_group = [0, 0, 0, 0, 0]

MAP_2018_long = [0, 0, 0, 0, 0]
MAP_2018_short = [0, 0, 0, 0, 0]
MAP_2018_seq = [0, 0, 0, 0, 0]
cutoff = 10

In [39]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)

evaluator_validation_earlystopping = EvaluatorHoldout(URM_validation, cutoff_list=[10])

earlystopping_keywargs = {"validation_every_n": 5,
                        "stop_on_validation": True,
                        "evaluator_object": evaluator_validation_earlystopping,
                        "lower_validations_allowed": 5,
                        "validation_metric": "MAP" ,
                        }

recommender_als = IALSRecommender(URM_train)
recommender_als.fit(epochs = 135,
        num_factors = 153,
        confidence_scaling = "log",
        alpha = 30.61917675327107,
        epsilon = 0.001001609694823079,
        reg = 1.3918031693071822e-05,
        validation_every_n = 5,
        stop_on_validation = True,
        evaluator_object = evaluator_validation_earlystopping,
        lower_validations_allowed = 5,
        validation_metric = "MAP" )

IALSRecommender: URM Detected 52 (0.65 %) cold users.
IALSRecommender: URM Detected 2033 (7.83 %) cold items.
IALSRecommender: Epoch 1 of 135. Elapsed time 21.54 sec
IALSRecommender: Epoch 2 of 135. Elapsed time 42.41 sec
IALSRecommender: Epoch 3 of 135. Elapsed time 1.06 min
IALSRecommender: Epoch 4 of 135. Elapsed time 1.43 min
IALSRecommender: Validation begins...
Deallocating Cython objects
EvaluatorHoldout: Processed 5079 ( 100.00% ) in 5.02 sec. Users per second: 1013
IALSRecommender: CUTOFF: 10 - ROC_AUC: 0.1326322, PRECISION: 0.0262650, PRECISION_RECALL_MIN_DEN: 0.1063955, RECALL: 0.1046472, MAP: 0.0479428, MRR: 0.0950682, NDCG: 0.0726364, F1: 0.0419909, HIT_RATE: 0.2626501, ARHR: 0.1056392, NOVELTY: 0.0049298, AVERAGE_POPULARITY: 0.0619067, DIVERSITY_MEAN_INTER_LIST: 0.9959800, DIVERSITY_HERFINDAHL: 0.9995784, COVERAGE_ITEM: 0.4671415, COVERAGE_ITEM_CORRECT: 0.0386526, COVERAGE_USER: 0.6391091, COVERAGE_USER_CORRECT: 0.1342645, DIVERSITY_GINI: 0.1884536, SHANNON_ENTROPY: 12.40

In [42]:
from KNN.ItemKNNScoresHybrid5Recommender import ItemKNNScoresHybrid5Recommender

In [43]:
for index in range(0, 5):
        URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)

        # ----- FIRST ------ #
        recommender_alpha = P3alphaRecommender(URM_train)
        recommender_alpha.fit(topK=170, alpha = 0.45, implicit = True)

        recommender_beta_ICM = RP3betaRecommender(ICM_all.T)
        recommender_beta_ICM.fit(topK=70, alpha=0.2, beta=0.5, implicit=False)
        recommender_beta_ICM.URM_train = URM_train

        hybridrecommender = ItemKNNSimilarityHybridRecommender(URM_train, recommender_alpha.W_sparse, recommender_beta_ICM.W_sparse)
        hybridrecommender.fit(topK=200, alpha = 0.1)


        # ----- SECOND  ------ #
        hybridrecommender_with_als = ItemKNNScoresHybridMultipleRecommender(URM_train, recommender_als, recommender_alpha, recommender_beta_ICM)
        hybridrecommender_with_als.fit(alpha = 0.3, beta=0.1)

        # ----- THIRD ------ #
        recommender_userKNN_top_model = UserKNNCFRecommender(URM_train)
        recommender_userKNN_top_model.fit(topK= 75, shrink = 200)

        recommender_SVD_top_model = PureSVDRecommender(URM_train)
        recommender_SVD_top_model.fit(num_factors = 1200)

        recommender_alpha_top_model = P3alphaRecommender(URM_train)
        recommender_alpha_top_model.fit(topK=350, alpha = 0.45, implicit = True)

        recommender_beta_ICM_top_model = RP3betaRecommender(ICM_all.T)
        recommender_beta_ICM_top_model.fit(topK=70, alpha=0.35, beta=0.4, implicit=False)
        recommender_beta_ICM_top_model.URM_train = URM_train

        hybridrecommender_p3_rp3_top_model = ItemKNNSimilarityHybridRecommender(URM_train, recommender_alpha_top_model.W_sparse, recommender_beta_ICM_top_model.W_sparse)
        hybridrecommender_p3_rp3_top_model.fit(topK=450, alpha = 0.1)

        hybridrecommender_top_model = ItemKNNScoresHybridMultipleRecommender(URM_train, hybridrecommender_p3_rp3_top_model, recommender_userKNN_top_model, recommender_SVD_top_model)
        hybridrecommender_top_model.fit(alpha = 0.7, beta=0.25)


        # ------ FORTH COPY FROM 2018 REPO ------ #
        # SHORT ---- LONG ----- SEQ

        recommender_slim_bpr_2018 = SLIM_BPR_Cython(URM_train, recompile_cython=False)
        recommender_slim_bpr_2018.fit(epochs=200, batch_size=1000, sgd_mode='adagrad', learning_rate=0.001, positive_threshold_BPR=1, topK=400)

        recommender_user_cf_2018 = UserKNNCFRecommender(URM_train)
        recommender_user_cf_2018.fit(shrink=100, topK=150, feature_weighting = "TF-IDF")

        recommender_item_cf_2018 = ItemKNNCFRecommender(URM_train)
        recommender_item_cf_2018.fit(shrink=200, topK=750, feature_weighting = 'TF-IDF', similarity = 'tversky')

        recommender_cbf_2018 = ItemKNNCBFRecommender(URM_train, ICM_all)
        recommender_cbf_2018.fit(topK=600, shrink=200, feature_weighting = 'TF-IDF')

        hybridrecommender_2018_short = ItemKNNScoresHybrid5Recommender(URM_train, recommender_user_cf_2018, recommender_item_cf_2018, recommender_cbf_2018, recommender_als, recommender_slim_bpr_2018)
        hybridrecommender_2018_short.fit(first = 0.03, second=0.25, third=0.15, forth=0.3, fifth=0.6)

        hybridrecommender_2018_long = ItemKNNScoresHybrid5Recommender(URM_train, recommender_user_cf_2018, recommender_item_cf_2018, recommender_cbf_2018, recommender_als, recommender_slim_bpr_2018)
        hybridrecommender_2018_long.fit(first = 0.03, second=0.35, third=0.2, forth=0.3, fifth=0.22)

        hybridrecommender_2018_seq = ItemKNNScoresHybrid5Recommender(URM_train, recommender_user_cf_2018, recommender_item_cf_2018, recommender_cbf_2018, recommender_als, recommender_slim_bpr_2018)
        hybridrecommender_2018_seq.fit(first = 0.5, second=0.1, third=0.72, forth=0.14, fifth=2.065)

        grouped_users = dataset.groupby(['row']).count()
        sorted_users = grouped_users.sort_values(by=['col'], ascending=True)
        sorted_users = sorted_users.index.to_numpy()

        block_size = int(len(sorted_users)*0.20)

        for group_id in range(0, 5):
        
                start_pos = group_id*block_size
                end_pos = min((group_id+1)*block_size, len(sorted_users))
                
                users_in_group = sorted_users[start_pos:end_pos]
                
                users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert = True)
                users_not_in_group = sorted_users[users_not_in_group_flag]
                
                evaluator_p3_rp3 = EvaluatorHoldout(URM_validation, cutoff_list=[cutoff], ignore_users = users_not_in_group)
                
                results, _ = evaluator_p3_rp3.evaluateRecommender(hybridrecommender)
                MAP_p3aplha_rp3beta_per_group[group_id] += results[cutoff]["MAP"]

                results, _ = evaluator_p3_rp3.evaluateRecommender(hybridrecommender_with_als)
                MAP_p3aplha_rp3beta_with_als_per_group[group_id] += results[cutoff]["MAP"]

                results, _ = evaluator_p3_rp3.evaluateRecommender(hybridrecommender_top_model)
                MAP_top_model_per_group[group_id] += results[cutoff]["MAP"]

                results, _ = evaluator_p3_rp3.evaluateRecommender(hybridrecommender_2018_long)
                MAP_2018_long[group_id] += results[cutoff]["MAP"]

                results, _ = evaluator_p3_rp3.evaluateRecommender(hybridrecommender_2018_short)
                MAP_2018_short[group_id] += results[cutoff]["MAP"]

                results, _ = evaluator_p3_rp3.evaluateRecommender(hybridrecommender_2018_seq)
                MAP_2018_seq[group_id] += results[cutoff]["MAP"]

Elapsed time 2.13 sec
Processed 7947 ( 100.00% ) in 0.17 seconds. BPR loss is 1.08E-03. Sample per second: 46577
SLIM_BPR_Recommender: Epoch 80 of 200. Elapsed time 2.15 sec
Processed 7947 ( 100.00% ) in 0.19 seconds. BPR loss is 1.14E-03. Sample per second: 41109
SLIM_BPR_Recommender: Epoch 81 of 200. Elapsed time 2.17 sec
Processed 7947 ( 100.00% ) in 0.22 seconds. BPR loss is 1.17E-03. Sample per second: 36527
SLIM_BPR_Recommender: Epoch 82 of 200. Elapsed time 2.20 sec
Processed 7947 ( 100.00% ) in 0.24 seconds. BPR loss is 1.30E-03. Sample per second: 32745
SLIM_BPR_Recommender: Epoch 83 of 200. Elapsed time 2.22 sec
Processed 7947 ( 100.00% ) in 0.27 seconds. BPR loss is 1.13E-03. Sample per second: 29430
SLIM_BPR_Recommender: Epoch 84 of 200. Elapsed time 2.25 sec
Processed 7947 ( 100.00% ) in 0.30 seconds. BPR loss is 1.21E-03. Sample per second: 26797
SLIM_BPR_Recommender: Epoch 85 of 200. Elapsed time 2.28 sec
Processed 7947 ( 100.00% ) in 0.32 seconds. BPR loss is 1.23E-03. 

AttributeError: 'ItemKNNScoresHybrid5Recommender' object has no attribute 'firth'

In [ ]:
for index in range(0, 5):
    MAP_p3aplha_rp3beta_per_group[index] /= 5
    MAP_p3aplha_rp3beta_with_als_per_group[index] /= 5
    MAP_top_model_als_per_group[index] /= 5
    MAP_top_model_per_group[index] /= 5
    MAP_2018_long[index] /= 5
    MAP_2018_short[index] /= 5
    MAP_2018_seq[index] /= 5

In [ ]:
import matplotlib.pyplot as pyplot
%matplotlib inline  
pyplot.figure(figsize=(15,10))
pyplot.plot(MAP_p3aplha_rp3beta_per_group, label="P3Aplha - RP3Beta")
pyplot.plot(MAP_p3aplha_rp3beta_with_als_per_group, label="P3Aplha - RP3Beta - ALS")
pyplot.plot(MAP_top_model_als_per_group, label="topModel als :O")
pyplot.plot(MAP_top_model_per_group, label="topModel :O")
pyplot.plot(MAP_2018_long, label="2018 long")
pyplot.plot(MAP_2018_short, label="2018 short")
pyplot.plot(MAP_2018_seq, label="2018 seq")

pyplot.ylabel('MAP')
pyplot.xlabel('User Group')
pyplot.legend()
pyplot.show()